## Hangman Algorithm

In [112]:
# Load modules 
from collections import Counter

In [113]:
# Load english word dictonary 
with open('words.txt') as f:
    english_dict = f.read().splitlines()
english_dict = list(set([x.lower() for x in english_dict]))

In [114]:
def pre_process_input(inputword):
    outputword = str(inputword).lower()
    return outputword

In [115]:
## more sophisticated might weight distribution by actual use/commonly known words 
def get_words_with_len(word_length):
    start_words = [word for word in english_dict if len(word) == word_length]
    return start_words

In [116]:
def get_letter_dist_from_word_list(list_of_words):
    all_letters = [list(w) for w in list_of_words]
    all_letters_flat = [item for sublist in all_letters for item in sublist]
    letter_count = Counter(all_letters_flat)
    return letter_count

In [117]:
def get_word_so_far(correct_guess_dict):
    word_so_far = []
    for i in range(0,input_word_length):
        if i in correct_guess_dict.keys():
            word_so_far.append(correct_guess_dict[i])
        else:
            word_so_far.append("_")
    word_so_far = str(word_so_far)
    return word_so_far

In [118]:
## SEARCH BY DISTRIBUTION TO FIND FIRST LETTER 
def guess_using_dist(dist):
    global guesses_list
    global num_misses
    global num_guesses
    global correct_guess_dict
    global dists_used 
    
    dist = [x for x in ordered_letter_dist if x[0] not in guesses_list]  # remove guesses already made
    letter_freq_tuple = dist[0] # get the largest letter frequency  
    letter = letter_freq_tuple[0] # get letter only 
    print("LETTER GUESSED: ", letter)
    guesses_list.append(letter)
    num_guesses += 1
    match_found = False 
    for index, position in enumerate(inputword):
        if letter == position:
            correct_guess_dict[index] = letter
            match_found = True 
            print("MATCH FOUND")
            print("NUMBER OF MISSES", num_misses)
            print("WORD SO FAR", get_word_so_far(correct_guess_dict))
            print(" ")
    if match_found == False:
        print("NO MATCH FOUND")
        num_misses += 1
        print("NUMBER OF MISSES:", num_misses)
        print("WORD SO FAR", get_word_so_far(correct_guess_dict))
        print(" ")
    return correct_guess_dict, guesses_list, num_guesses, num_misses 

In [119]:
## RECALC DISTRIBUTION WITH NEW WORD INFO 
def get_remaining_words(word_list_in, correct_guess_dict,guesses_list):

    remain_words = word_list_in 
    
    # remove words with letters guessed which are wrong and therefore not in it 
    wrong_letters = [letter for letter in guesses_list if letter not in list(correct_guess_dict.values())]
    remain_words_out1 = remain_words 
    for guess_letter in wrong_letters:
        remain_words_out1 = [word for word in remain_words if guess_letter not in word]

    # keep only words with correctly guessed letters in the same position 
    remain_words_out2 = remain_words_out1
    for i in correct_guess_dict.keys():
        correct_letter = correct_guess_dict[i]
        remain_words_out2 = [word for word in remain_words_out1 if correct_letter == word[i]]
        
    print("NUMBER OF WORDS REMAINING", len(remain_words_out2))
    if len(remain_words_out2) < 11:
        print(remain_words_out2)
    return remain_words_out2


## Play game

In [136]:
## INITIATE GAME METRICS
guesses_list = []
num_guesses = 0
num_misses = 0
correct_guess_dict = {}
game_over = False 
finalword = ''

In [137]:
## input word 
inputword = 'marvin'
#inputword = input()

if len([x for x in english_dict if inputword in x]) == 0:
    print("Word is not in the dictonary, try another")

In [138]:
## GET INPUT WORD LENGTH
inputword = pre_process_input(inputword)
input_word_length = len(inputword)

In [140]:
start_words = get_words_with_len(input_word_length)
while len(start_words) > 1:
    letter_dist = get_letter_dist_from_word_list(start_words)
    ordered_letter_dist = letter_dist.most_common()

    correct_guess_dict, guesses_list, num_guesses, num_misses = guess_using_dist(ordered_letter_dist)
    start_words = get_remaining_words(start_words, correct_guess_dict,guesses_list)
else:
    print("GAME OVER", start_words)
    print("FINAL NUMBER OF MISSES", num_misses)


LETTER GUESSED:  e
NO MATCH FOUND
NUMBER OF MISSES: 1
WORD SO FAR ['_', '_', '_', '_', '_', '_']
 
NUMBER OF WORDS REMAINING 17752
LETTER GUESSED:  a
MATCH FOUND
NUMBER OF MISSES 1
WORD SO FAR ['_', 'a', '_', '_', '_', '_']
 
NUMBER OF WORDS REMAINING 4337
LETTER GUESSED:  n
MATCH FOUND
NUMBER OF MISSES 1
WORD SO FAR ['_', 'a', '_', '_', '_', 'n']
 
NUMBER OF WORDS REMAINING 644
LETTER GUESSED:  o
NO MATCH FOUND
NUMBER OF MISSES: 2
WORD SO FAR ['_', 'a', '_', '_', '_', 'n']
 
NUMBER OF WORDS REMAINING 395
LETTER GUESSED:  i
MATCH FOUND
NUMBER OF MISSES 2
WORD SO FAR ['_', 'a', '_', '_', 'i', 'n']
 
NUMBER OF WORDS REMAINING 148
LETTER GUESSED:  r
MATCH FOUND
NUMBER OF MISSES 2
WORD SO FAR ['_', 'a', 'r', '_', 'i', 'n']
 
NUMBER OF WORDS REMAINING 33
LETTER GUESSED:  m
MATCH FOUND
NUMBER OF MISSES 2
WORD SO FAR ['m', 'a', 'r', '_', 'i', 'n']
 
NUMBER OF WORDS REMAINING 7
['margin', 'martin', 'marrin', 'marvin', 'marwin', 'marlin', 'marcin']
LETTER GUESSED:  g
NO MATCH FOUND
NUMBER OF MI

In [80]:
## TODO

# double same letter bug [X]
# find better dictonary with plurals etc.  [x]
# could it be improved by guesses when small number of words left even at cost of a miss? 
# remove stopwords?
# weight words by commonality 
